In [2]:
from PIL import Image
import os,glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./catdataset/train"
categories = ["Bengal", "Russianblue","Bombay","ragdoll"]  ##카테고리 설정
nb_classes = len(categories)

image_w=128
image_h=128

pixels=image_h*image_w*3

X=[]
y=[]

for idx, cat in enumerate(categories):
    
    label=[0 for i in range(nb_classes)]
    label[idx]=1
    
    image_dir =caltech_dir +"/"+cat
    files=glob.glob(image_dir+"/*.jpg")
    print(cat,"파일 길이:",len(files))
    for i,f in enumerate(files):
        img = Image.open(f)
        img=img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)   
        y.append(label)
        
        if i % 700 == 0:   ##???
            print(cat, " : ", f)

X = np.array(X) ##배열 
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)   ##그래프 표현을 위한 설정
xy = (X_train, X_test, y_train, y_test)
np.save("./numpy_data/m_image_data.npy", xy)  

print("ok", len(y))


Bengal 파일 길이: 9996
Bengal  :  ./catdataset/train/Bengal\Bengal (1).jpg
Bengal  :  ./catdataset/train/Bengal\Bengal (717).jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_1368.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_2015.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_2666.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_3317.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_3966.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_4611.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_5256.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_5906.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_6550.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_7193.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_7837.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_8481.jpg
Bengal  :  ./catdataset/train/Bengal\Bengal_0_9125.jpg
Russianblue 파일 길이: 10000
Russianblue  :  ./catdataset/train/Russianblue\Russian (1).jpg
Russianblue  :  ./catdataset/train/Russianblue\Russian (729).jpg
Russian

In [1]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
   

import tensorflow as tf   #GPU 구현에 있어서 메모리를 절약해서 사용하기 위한 코드
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

X_train, X_test, y_train, y_test = np.load('./numpy_data/m_image_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

(29996, 128, 128, 3)
29996


In [2]:
import numpy as np
import pandas as pd
categories = ["Bombay", "Russianblue","Bengal","ragdoll"]  ##카데고리 설정
nb_classes = len(categories)

X_train = X_train.astype('float64')/255
X_test = X_test.astype('float64')/255

In [5]:

    model = Sequential()##선형계층
    model.add(Conv2D(16, kernel_size=(3, 3),padding="same", activation='relu', input_shape=X_train.shape[1:]))
    model.add(Conv2D(32, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.125))

    model.add(Conv2D(32, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))
    
    model.add(Conv2D(64, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))
    
    model.add(Conv2D(128, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))
    
    model.add(Conv2D(256, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(Conv2D(512, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))
    
    model.add(Conv2D(512, kernel_size=(3, 3),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))
    
              
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(nb_classes, activation='softmax')) ##soft max->0부터1사이의 출력값은 실수이며 출력의 총합을 1로 표현 확률을 표현할때 사용
   
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) ##optimization ->딥러닝에서 학습속도를 빠르고 안정적이게 하는것  
    model_dir = './model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/my_classification.h5'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True) ##keras 모델 체크포인트 save_best_only->정확도의 최고값을 갱신할때만 사용
    early_stopping = EarlyStopping(monitor='val_loss', patience=10) ##학습 종료 patience 성능이 증가하지 않는 epoch를 몇번이나 허용할 것인가 정의
    ##최적의 데이터를 학습하면 더이상 학습을 하지 않는다
    

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0

In [6]:

history = model.fit(X_train, y_train, batch_size=256, epochs=50, validation_split=0.2, callbacks=[checkpoint, early_stopping])


Epoch 1/50
94/94 [==============================] - ETA: 0s - loss: 1.3457 - accuracy: 0.2945 
Epoch 00001: val_loss improved from inf to 1.11120, saving model to ./model\my_classification.h5
94/94 [==============================] - 1019s 11s/step - loss: 1.3457 - accuracy: 0.2945 - val_loss: 1.1112 - val_accuracy: 0.4425
Epoch 2/50
94/94 [==============================] - ETA: 0s - loss: 0.8807 - accuracy: 0.5580 
Epoch 00002: val_loss improved from 1.11120 to 0.55428, saving model to ./model\my_classification.h5
94/94 [==============================] - 989s 11s/step - loss: 0.8807 - accuracy: 0.5580 - val_loss: 0.5543 - val_accuracy: 0.7258
Epoch 3/50
94/94 [==============================] - ETA: 0s - loss: 0.5377 - accuracy: 0.7448 
Epoch 00003: val_loss improved from 0.55428 to 0.41340, saving model to ./model\my_classification.h5
94/94 [==============================] - 985s 10s/step - loss: 0.5377 - accuracy: 0.7448 - val_loss: 0.4134 - val_accuracy: 0.8230
Epoch 4/50
94/94 [====

In [7]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 70s 225ms/step - loss: 0.0776 - accuracy: 0.9828
정확도 : 0.9828
